In [57]:
import numpy as np
import pandas as pd

In [100]:
import yfinance as yf
tickers = ['AGG','SPY','GLD','HYG']
    
data = yf.download(tickers, start="2015-01-01", end="2020-01-01")

[*********************100%***********************]  4 of 4 completed


In [101]:
prices = data['2018':'2020']['Adj Close'][tickers]
prices.to_csv('prices.csv')

# PyPortfolioOpt

In [102]:
# !pip install PyPortfolioOpt

# Import the packages 
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.efficient_frontier import EfficientFrontier

In [103]:
prices = pd.read_csv("prices.csv", parse_dates=['Date'], index_col='Date')

In [104]:
# Calculate expected returns
expected_returns_daily = expected_returns.mean_historical_return(prices)
expected_returns_daily

AGG    0.043622
SPY    0.120588
GLD    0.072331
HYG    0.056586
dtype: float64

In [105]:
# Calculate the covariance matrix
covariance_matrix = risk_models.sample_cov(prices)

In [106]:
# Obtain the efficient frontier
ef = EfficientFrontier(expected_returns_daily, covariance_matrix)

In [107]:
# Select a chosen optimal portfolio
ef.max_sharpe(risk_free_rate=0.00)
print('Weights: {}'.format(ef.clean_weights()))
# Obtain the performance numbers
ef.portfolio_performance(verbose=True, risk_free_rate = 0.00)

Weights: {'AGG': 0.71394, 'SPY': 0.03533, 'GLD': 0.0, 'HYG': 0.25073}
Expected annual return: 5.0%
Annual volatility: 2.5%
Sharpe Ratio: 1.96


(0.04959225286374169, 0.02533414791276617, 1.9575259856579421)

In [108]:
# Select a chosen portfolio with minimal volatiliy
ef.min_volatility()
print('Weights: {}'.format(ef.clean_weights()))
# Obtain the performance numbers
ef.portfolio_performance(verbose=True, risk_free_rate = 0.00)

Weights: {'AGG': 0.58031, 'SPY': 0.0, 'GLD': 0.0, 'HYG': 0.41969}
Expected annual return: 4.9%
Annual volatility: 2.6%
Sharpe Ratio: 1.91


(0.049063192755046475, 0.025729992079657774, 1.906848342710374)

In [109]:
def genEFdata(ef, vsteps=0.001, rf=0.00):
    
    RISK_FREE_RATE = rf

    results = pd.DataFrame([])

    min_vol = ef.min_volatility()
    perf = ef.portfolio_performance(verbose=False, risk_free_rate = RISK_FREE_RATE)
    perf_dict = {}
    perf_dict['index'] = 'min_vol'
    perf_dict['annual_return'] = perf[0]
    perf_dict['annual_volatility'] = perf[1]
    perf_dict['sharpe_ratio'] = perf[2]
    results = results.append(perf_dict, ignore_index=True)
    lower_bound = perf[1]

    max_sharpe = ef.max_sharpe(risk_free_rate = RISK_FREE_RATE)
    perf = ef.portfolio_performance(verbose=False, risk_free_rate = RISK_FREE_RATE)
    perf_dict = {}
    perf_dict['index'] = 'max_sharpe'
    perf_dict['annual_return'] = perf[0]
    perf_dict['annual_volatility'] = perf[1]
    perf_dict['sharpe_ratio'] = perf[2]
    results = results.append(perf_dict, ignore_index=True)
    upper_bound = perf[1] * 2

    count = 0
    
    lower_bound = 0
    for vol in np.arange(lower_bound, upper_bound, vsteps):
        try:
            wgt = ef.efficient_risk(vol, risk_free_rate = RISK_FREE_RATE)
            perf = ef.portfolio_performance(verbose=False, risk_free_rate = RISK_FREE_RATE)

            count += 1
            perf_dict = {}
            perf_dict['index'] = 'calc_' + str(count)
            perf_dict['annual_return'] = perf[0]
            perf_dict['annual_volatility'] = perf[1]
            perf_dict['sharpe_ratio'] = perf[2]

            results = results.append(perf_dict, ignore_index=True)
        except:
            continue
            
    return results.set_index('index')

In [110]:
ef_data = genEFdata(ef, vsteps=0.0005)

In [111]:
ef_data.sort_values('sharpe_ratio').tail()

,annual_return,annual_volatility,sharpe_ratio
index,,,
calc_5,0.052312,0.027000,1.937499
calc_4,0.051576,0.026500,1.946276
calc_3,0.050782,0.026000,1.953146
calc_2,0.049907,0.025500,1.957146
max_sharpe,0.049592,0.025334,1.957526


In [112]:
ef_data.sort_values('annual_volatility').head()

,annual_return,annual_volatility,sharpe_ratio
index,,,
calc_1,0.047441,0.024500,1.936377
max_sharpe,0.049592,0.025334,1.957526
calc_2,0.049907,0.025500,1.957146
min_vol,0.049063,0.025730,1.906848
calc_3,0.050782,0.026000,1.953146
